In [ ]:
cfg_id_column = 0
cfg_title_column = 2
cfg_description_column = 13
cfg_depend_column = 4
cfg_first_valid_row = 1
cfg_project_prefix_cfid = 21
cfg_rmtracker_id = 3
cfg_rmtitle_cfid = 16
cfg_enable_duration = True
cfg_enable_assignee = True
cfg_duration_column = 9
cfg_assignee_column = 11
cfg_enable_collaborators = True
cfg_collaborators_column = 12
rm_cfield_collaborators = 29
cfg_enable_workload = False
cfg_workload_column = 10
rm_cfield_dep = 32
cfg_enable_otherdep = True
cfg_otherdep_column = 5

In [ ]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

In [ ]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
projects = redmine.project.all()

In [ ]:
print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

In [ ]:
file_path = "input_dsm.ods"
data = get_data(file_path)['_RqDep']
tasks = {}

In [ ]:

i = 0
tasks = {}
for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        thisTsk = {}
        if (len(row) > cfg_title_column) and (len(row[cfg_title_column]) > 0):
            thisTsk['rmid'] = 0
            thisTsk['rmident'] = ''
            thisTsk['row'] = i
            thisTsk['ident'] = row[cfg_id_column].strip()
            thisTsk['title'] = row[cfg_title_column].strip()
            if (len(row) > cfg_description_column):
                thisTsk['description'] = row[cfg_description_column].strip()
            else:
                thisTsk['description'] = None
            thisTsk['dependences'] = []
            if (len(row) > cfg_depend_column):
                tmp = row[cfg_depend_column].strip()
                if (tmp != ''):
                    thisTsk['dependences'] = tmp.split(' ')
                                
            if (cfg_enable_otherdep):
                thisTsk['otherdep'] = []
                if (len(row) > cfg_otherdep_column):
                    tmp = row[cfg_otherdep_column].strip()
                    if (tmp != ''):
                        thisTsk['otherdep'] = tmp.split(' ')
            
            #print(thisTsk)
            tasks[thisTsk['ident']] = thisTsk
            print(thisTsk['ident'])
            
            if (cfg_enable_duration):
                if (len(row) > cfg_duration_column):
                    tmp = row[cfg_duration_column]
                    if (tmp is not None) and (tmp != ''):
                        thisTsk['duration'] = tmp
                    
            if (cfg_enable_assignee):
                if (len(row) > cfg_assignee_column):
                    tmp = row[cfg_assignee_column].strip()
                    if (tmp is not None) and (tmp != ''):
                        print("assignee: |"+tmp+"|")
                        thisTsk['assignee'] = tmp                    

            if (cfg_enable_collaborators):
                thisTsk['collaborators'] = []
                if (len(row) > cfg_collaborators_column):
                    tmp = row[cfg_collaborators_column].strip()
                    if (tmp is not None) and (tmp != ''):
                        print("collaborators: |"+tmp+"|")
                        thisTsk['collaborators'] = tmp.split(' ')
                        
                print(thisTsk['collaborators'])


    
    i += 1
    


In [ ]:
print(tasks.keys())

Vamos a buscar las mismas tareas al Redmine, si es posible encontrarlas

In [ ]:
# Buscamos el prefijo del proyecto
prjPrefix = my_project.custom_fields.get(cfg_project_prefix_cfid).value
lenPrjPrefix = len(prjPrefix)
# Recorreremos los identificadores de las tareas cargadas del ODS para ver si alguno se corresponde con un identificador válido de proyecto
for tskey in tasks.keys():
    thisRmTsk = None
    if (len(tskey)>lenPrjPrefix+1):
        tskPrefix = tskey[0:lenPrjPrefix]
        if (tskPrefix == prjPrefix) and (tskey[lenPrjPrefix] == '-'):
            print("Identificador rm:",tskey)
            thisRmTskList = myproject.issues.filter(status_id='*',subject=tskey)
            if (len(thisRmTskList)==1):
                # hemos encontrado una tarea de Redmine que cuadra
                thisRmTsk = thisRmTskList[0]
                tasks[tskey]['rmident'] = tskey
                tasks[tskey]['rmid'] = thisRmTsk.id
                tasks[tskey]['rmtsk'] = thisRmTsk
                
    if (thisRmTsk == None):
        print("Tarea nueva",tskey)
        thisRmTsk = redmine.issue.create(project_id = my_project.id,
            tracker_id = cfg_rmtracker_id,
            description = tasks[tskey]['description'],
            custom_fields=[
                {'id': cfg_rmtitle_cfid,'value': tasks[tskey]['title']}
            ]
        )
        tasks[tskey]['rmident'] = thisRmTsk.subject
        tasks[tskey]['rmid'] = thisRmTsk.id
        tasks[tskey]['rmtsk'] = thisRmTsk


In [ ]:
for tskey in tasks.keys():
    print("id",tskey,"rmident",tasks[tskey]['rmident'])

In [ ]:
# Recorreremos de nuevo los identificadores de las tareas cargadas del ODS para crear/modificar sus relaciones
for tskey in tasks.keys():
    odsDependences = tasks[tskey]['dependences']
    print("id",tskey,odsDependences)
    rmDependences = []
    for dpd in odsDependences:
        if (dpd != ''):
            thisRmId = tasks[dpd]['rmid']
            print(thisRmId)
            rmDependences.append(thisRmId)
        
    tasks[tskey]['rmdep'] = rmDependences
    

In [ ]:
if (cfg_enable_otherdep):
    for tskey in tasks.keys():
        odsOtherDep = tasks[tskey]['otherdep']
        print("id",tskey,odsOtherDep)
        rmOtherDep = []
        for dpd in odsOtherDep:
            if (dpd != ''):
                thisRmId = tasks[dpd]['rmid']
                print(thisRmId)
                rmOtherDep.append(thisRmId)

        tasks[tskey]['rmotherdep'] = rmOtherDep    

In [ ]:
# Ahora buscamos las dependencias ya existentes en el Redmine y las restamos de las relaciones a añadir
for tskey in tasks.keys():
    print("id",tskey,tasks[tskey]['dependences'],tasks[tskey]['rmdep'])
    thisTskRmId = tasks[tskey]['rmid']
    thisTskRelations = tasks[tskey]['rmtsk'].relations
    thisTskInputRelations = list(filter(lambda x: x.issue_to_id == thisTskRmId and x.relation_type=='precedes', thisTskRelations))
    print("Preexistent:", thisTskInputRelations)
    for rel in thisTskInputRelations:
        # La relación está a punto de ser añadida, cancelamos su creación
        if rel.issue_to_id in tasks[tskey]['rmdep']:
            tasks[tskey]['rmdep'].remove(rel.issue_to_id)
            print("En",tskey,"Borro",rel.issue_to_id,"de",tasks[tskey]['rmdep'])

        if rel.issue_to_id in tasks[tskey]['rmotherdep']:
            tasks[tskey]['rmotherdep'].remove(rel.issue_to_id)
            print("En",tskey,"Borro",rel.issue_to_id,"de",tasks[tskey]['rmotherdep'])               
        
        else:
            # La relación ya no existe, hay que borrarla
            redmine.issue_relation.delete(rel.id)
            
    # Ahora añadimos las relaciones restantes
    for relId in tasks[tskey]['rmdep']:
        rel = redmine.issue_relation.create(
            issue_id=relId,
            issue_to_id=thisTskRmId,
            relation_type='precedes',
            delay=0
        )

            


In [ ]:
# Ahora buscamos las dependencias ya existentes en el Redmine y las restamos de las relaciones a añadir
for tskey in tasks.keys():
    print("id",tskey,tasks[tskey]['otherdep'],tasks[tskey]['rmotherdep'])
    thisTskRmId = tasks[tskey]['rmid']            
    # Ahora añadimos las relaciones restantes
    for relId in tasks[tskey]['rmotherdep']:
        rel = redmine.issue_relation.create(
            issue_id=relId,
            issue_to_id=thisTskRmId,
            relation_type='precedes',
            delay=0
        )


In [ ]:
members = my_project.memberships
team = {}
for m in members:
    thisUser = redmine.user.get(m.user.id)
    team[thisUser.login] = thisUser
    
print(team.keys())


In [ ]:
if (cfg_enable_duration):    
    for tskey in tasks.keys():
        if 'duration' in tasks[tskey].keys():
            #print("------------>",tasks[tskey]['duration'])
            tasks[tskey]['rmtsk'].estimated_hours = tasks[tskey]['duration']
            tasks[tskey]['rmtsk'].save()


In [ ]:
if (cfg_enable_assignee):
    for tskey in tasks.keys():
        print(tasks[tskey].keys())
        print(tasks[tskey]['ident'])
        if ('assignee' in tasks[tskey].keys()):
            print(tasks[tskey]['assignee'])
            tasks[tskey]['rmtsk'].assigned_to_id = team[tasks[tskey]['assignee']].id
            print(tasks[tskey]['rmtsk'])
            tasks[tskey]['rmtsk'].save()

In [ ]:
if (cfg_enable_collaborators):
    for tskey in tasks.keys():
        if ('collaborators' in tasks[tskey].keys()):
            odsCollaborators = tasks[tskey]['collaborators']
            print("collaborators",tskey,odsCollaborators)
            rmCollaborators = []
            for col in odsCollaborators:
                if (col != ''):
                    colRmId = team[col].id
                    print(team[col].login)
                    rmCollaborators.append(colRmId)

            print(rmCollaborators)
            tasks[tskey]['rmcol'] = rmCollaborators
            collabCf = tasks[tskey]['rmtsk'].custom_fields.get(rm_cfield_collaborators).value
            print(collabCf)
            redmine.issue.update(resource_id=tasks[tskey]['rmtsk'].id,
                                 custom_fields=[{'id': rm_cfield_collaborators,'value': rmCollaborators}]
                                )


In [ ]:
for tskey in tasks.keys():
    depstr = ','.join(map(str,tasks[tskey]['rmdep']))
    print(depstr)
    depCf = tasks[tskey]['rmtsk'].custom_fields.get(rm_cfield_dep).value
    print(depCf)
    redmine.issue.update(resource_id=tasks[tskey]['rmtsk'].id,
                         custom_fields=[{'id': rm_cfield_dep,'value': depstr}]
                        )



In [ ]:
print("done")